# 1 Import Libraries and Packages 

In [ ]:
#Import the libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

from datetime import datetime, timedelta, date
from prophet import Prophet
from neuralprophet import NeuralProphet
from pmdarima.arima import auto_arima
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.exponential_smoothing import ets
from statsmodels.tools.eval_measures import rmse, mse
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
user = os.getenv("USER")

# 2 Import the dataset and check the columns

In [ ]:
#Import the dataset

data = pd.read_csv(f"/Users/{user}/forecasting-part-two/victoria_covid_cases_source_updated.csv", date_parser=True)

In [ ]:
print(data.shape) #Getting the shape of the data
data.head()

In [ ]:
#Getting all the dates between, since there is a chance some dates in between are missing
full_dates = pd.date_range(start=data.diagnosis_date.min(),
              end=data.diagnosis_date.max())
full_dates = pd.to_datetime(full_dates, format="%Y-%m-%d")
print(full_dates.shape)

In [ ]:
#Aggregating the data, and getting daily cases
vic_cases = data.groupby(["diagnosis_date"]).agg({"Postcode":"count"})\
                .rename(columns={"Postcode":"total_cases"})
#Formatting the index
vic_cases.index = pd.to_datetime(vic_cases.index, format='%Y-%m-%d')

#Filling the missing dates
vic_cases = vic_cases.reindex(full_dates,fill_value=0)

#print the shape
print(vic_cases.shape)

vic_cases.head()

In [ ]:
#updating the date from object datatype to date datatype 
vic_cases.index.asfreq = "d" #adding the frequency to daily

# 3 Explore the data

In [ ]:
#Chaging the figsize
plt.rcParams["figure.figsize"] = (12,8)

#Lets the plot the data
vic_cases.plot()
plt.ylabel("Num Cases");

In [ ]:
#Lets apply the Seasonal decomposition
results = seasonal_decompose(vic_cases["total_cases"], period=207)
results.plot();

# 4 Find the AR, MA, I values using plots and do some statistical tests

In [ ]:
#Lets check if our lags are correlated
total_lags = [1, 2, 5, 10, 20]
fig, axes = plt.subplots(nrows=len(total_lags), ncols=1, figsize=(12,40))
index=0
for lg in total_lags:
    pd.plotting.lag_plot(vic_cases["total_cases"], lag=lg, ax=axes[index]);
    axes[index].set_title(f"Correlation cases with lag {lg}")
    index+=1

In [ ]:
#Lets make it easier and use acf plot, and use this to get the value for AR side of the model
plot_acf(vic_cases['total_cases'],lags=30);

In [ ]:
#Lets make it easier and use pacf plot, and use this to get the value for MA side of the model
plot_pacf(vic_cases['total_cases'],lags=30);

In [ ]:
#Let check if data is stationary, if value is below 0.05, we can determine data is stationary
def adf_fuller_df(df, colname=""):
    """"The null hypothesis of the Augmented Dickey-Fuller is that there is a unit
        root, which means there is a trend component, with the alternative that 
        there is no unit root, hence no trend component. If the pvalue is above 
        a critical size, then we cannot reject that there is a unit root."""

    df = df.copy()
    vals = adfuller(df[colname])
    first_columns = ["adf", "p-value", "usedlags", "number of observations"]
    other_column = "icbest"
    df = pd.DataFrame(data=pd.Series(vals[0:4], index=first_columns)).T
    for key, val in vals[4].items():
        df[f"Critical value for {key}"] = val
    df[other_column] = vals[5]
    p_value = df["p-value"][0]
    if  p_value <= 0.05:
        print(f"""We should reject the null hypothesis, since there is enough 
        evidence series is stationary, as there is only {np.round(p_value*100,2)} percent
        likelihood, that series is not stationary, and is very small""")
    else:
        print("We should further difference the series with itself ")
    
    return df

In [ ]:
adf_fuller_df(vic_cases, "total_cases")

In [ ]:
#Can we use auto arima instead to get all values of AR, I and MA
# Based on the plots, we could say AR can be max 21, MA could be max 3 and I is 0

# 5 Run auto arima to find out values for AR, MA and I side of the modes

In [ ]:
auto_arima_model = auto_arima(vic_cases["total_cases"],
                               max_p=30, 
                               max_q=30, 
                               max_order=62, 
                               n_jobs=1, 
                               seasonal=False, 
                               random_state=42, #seed
                               n_fits=10,
                               trace=True
                             )
#Getting the summary from the model
auto_arima_model.summary()

# 6 Lets split the data into training and Test

In [ ]:
#Lets split the data into train and testing
train = vic_cases[:"2021-08-20"]
test = vic_cases["2021-08-21":]

In [ ]:
ax = train.plot()
ax.plot(test)
ax.legend(["TRAIN", "TEST"]);

In [ ]:
auto_arima_model = auto_arima(train["total_cases"],
                               max_p=30, 
                               max_q=30, 
                               max_order=62, 
                               n_jobs=1, 
                               seasonal=False, 
                               random_state=42, #seed
                               n_fits=10,
                               trace=True
                             )
#Getting the summary from the model
auto_arima_model.summary()

# 7 Train the model based on ARIMA terms from Auto arima

In [ ]:
#train the data based on the parameters we got
arima_model = ARIMA(train["total_cases"], order=(5,1,2)).fit()
arima_model.summary()

# 8 Make predictions

In [ ]:
#Predict the model
start = train.shape[0] 
end = train.shape[0] + test.shape[0] - 1
arima_predictions = arima_model.predict(start=start, end=end)

In [ ]:
results=test
test["predictions"] = [p for p in arima_predictions]

In [ ]:
#plotting the results
ax = train.plot()
ax.plot(test["total_cases"])
ax.plot(test["predictions"])
ax.set_xlim(date(2021,7,1),date(2021,8,25))
ax.set_ylim(-3,100)
ax.legend(["TRAIN", "TEST", "PRED"]);

# 9 Evaluate the model

In [ ]:
#measure performance
RMSE=rmse(results.total_cases, results.predictions)
MSE=mse(results.total_cases, results.predictions)

print(f"rmse is {RMSE}")
print(f"mse is {MSE}")
print("mean is {}".format(test.total_cases.mean()))

# 10 Lets try model building training, testing with Fb Prophet Library

In [ ]:
#The library like dates in DS column and target variable in Y
vic_cases = vic_cases.reset_index()
vic_cases = vic_cases.rename(columns={"total_cases":"y", "index":"ds"})
vic_cases.head()

In [ ]:
#splitting data into training and testing
trainX=vic_cases[:-test.shape[0]]
textX=vic_cases[-test.shape[0]:]

In [ ]:
#Train the model

#Define the model
fb_model=Prophet()

#Fit the model
fb_model.fit(trainX)

In [ ]:
#future dates
future = fb_model.make_future_dataframe(periods=test.shape[0])
future.tail(test.shape[0])

In [ ]:
#forecast
fb_predictions=fb_model.predict(future)
fb_predictions[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(test.shape[0])


In [ ]:
#plot the data
fig1 = fb_model.plot(fb_predictions)


In [ ]:
train.total_cases = train.total_cases.astype(float)
test.total_cases = test.total_cases.astype(float)

In [ ]:
ets_model = ets.ETSModel(train['total_cases'], freq='D', error='add').fit()
predictions_ets = ets_model.predict(start=start, end=end)

In [ ]:
fb_pred = fb_predictions[["ds","yhat"]][-test.shape[0]:]
fb_pred.set_index("ds", inplace=True)

In [ ]:
fb_pred

# 11 Create ensemble of three models and visualise the results

In [ ]:
#Ensemble predictions
ensemble = (fb_pred.yhat + predictions_ets + test.predictions)/3

In [ ]:
ax = train.plot()
ax.plot(test["total_cases"])
ax.plot(test["predictions"])
ax.plot(fb_pred['yhat'])
ax.plot(predictions_ets)
ax.plot(ensemble)
ax.set_xlim(date(2021,7,1),date(2021,8,23))
ax.set_ylim(-3,100)
ax.legend(["TRAIN", "TEST", "PRED_ARIMA","PRED_FB", "PRED_ETS","ENSEMBLE"]);

# 12 Add some exogenous variables to the model and try make better predictions

In [ ]:
#Adding some features
vic_cases_wf = data[["diagnosis_date","acquired","Postcode"]].pivot_table(index="diagnosis_date", columns="acquired", values="Postcode", aggfunc="count", fill_value=0)
vic_cases_wf["Community"] = vic_cases_wf["Contact with a confirmed case"] + vic_cases_wf["Under investigation"]
vic_cases_wf["lockdown"] = vic_cases_wf.Community.apply(lambda val: 1 if val >= 10 else 0)

#Formatting the index
vic_cases_wf.index = pd.to_datetime(vic_cases_wf.index, format='%Y-%m-%d')

#Filling the missing dates
vic_cases_wf = vic_cases_wf.reindex(full_dates,fill_value=0)
vic_cases_wf.index.rename("index", inplace=True)

#Adding the delta feature
vic_cases_wf["delta"] = vic_cases_wf.index.map(lambda val : 1 if val >= date(2021,6,1) else 0 )

#print the shape
print(vic_cases_wf.shape)

vic_cases_wf.head()

In [ ]:
#Lets visualise number of common cases
vic_cases_wf.Community.plot.hist(bins=100)
plt.xlim(0,100);

In [ ]:
#Plotting the lockdown
vic_cases_wf.lockdown.plot();

In [ ]:
#Plotting delta
vic_cases_wf.delta.plot();

In [ ]:
#features to include in the model
features = ["delta","lockdown"]

In [ ]:
#Subsetting the data
vic_cases_wf_final = vic_cases_wf[features]
vic_cases_wf_final = vic_cases_wf_final.reset_index()\
                                       .merge(vic_cases, left_on="index", right_on="ds")\
                                       .drop(["index"],1)
vic_cases_wf_final.set_index("ds", inplace=True)
vic_cases_wf_final.head()

In [ ]:
#Lets split the data into train and testing
train = vic_cases_wf_final[:"2021-08-20"]
test = vic_cases_wf_final["2021-08-21":]

In [ ]:
auto_arima_model = auto_arima(train["y"],
                               X=train[features],
                               max_p=30, 
                               max_q=30, 
                               max_order=62, 
                               n_jobs=1, 
                               seasonal=True, 
                               random_state=42, #seed
                               n_fits=10,
                               m=7,
                               trace=True
                             )
#Getting the summary from the model
auto_arima_model.summary()

In [ ]:
auto_arima_model = auto_arima(train["y"],
                               X=train[features],
                               max_p=30, 
                               max_q=30, 
                               max_order=62, 
                               n_jobs=1, 
                               seasonal=False, 
                               random_state=42, #seed
                               n_fits=10,
                               trace=True
                             )
#Getting the summary from the model
auto_arima_model.summary()

In [ ]:
#Train the models
sarmia_model_exog=SARIMAX(train["y"], exog=train[features],order=(3,0,1), seasonal_order=(2,0,0,7), freq="D", max_iter=200).fit()
arima_model_exog=ARIMA(train["y"], exog=train[features], order=(1,0,5)).fit()


In [ ]:
sarima_exog_pred=sarmia_model_exog.predict(start=start, end=end, exog=test[features])
arima_exog_pred=arima_model_exog.predict(start=start, end=end, exog=test[features])

# 13 Visualise the predictions against actual data after retraining with variables

In [ ]:
ax = train["y"].plot()
ax.plot(test["y"])
ax.plot(sarima_exog_pred)
ax.plot(arima_exog_pred)
ax.set_xlim(date(2021,7,1),date(2021,8,25))
ax.set_ylim(-3,100)
ax.legend(["TRAIN", "TEST", "PRED_EXOG_SARIMA","PRED_EXOG_ARIMA"]);

# 14 Adding variables to FB model and predicting in the future

In [ ]:
#Prepare training and testing data for FB prophet
train = train.reset_index()
test = test.reset_index()
fb_train = train[["ds","y"] + features]
fb_test = test[["ds","y"] + features]

In [ ]:
#Traing fb with exog variables
fb_model_exog = Prophet()
for _ in features:
    fb_model_exog.add_regressor(_)
fb_model_exog.fit(fb_train)

In [ ]:
#Lets make predictions for the model
fb_exog_future = fb_model_exog.make_future_dataframe(periods=test.shape[0])
for _ in features:
    fb_exog_future[_] = [val for val in vic_cases_wf_final[_]]
fb_exog_forecast = fb_model_exog.predict(fb_exog_future)
#forecast
fb_exog_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(test.shape[0])

In [ ]:
#plot the data
fig1 = fb_model_exog.plot(fb_exog_forecast)

In [ ]:
#Ensemble predictions
fb_exog_pred = fb_exog_forecast[["ds","yhat"]][-test.shape[0]:]
fb_exog_pred.set_index("ds", inplace=True)
ensemble_exog = (fb_exog_pred.yhat + sarima_exog_pred + arima_exog_pred)/3
train.set_index("ds", inplace=True)
test.set_index("ds", inplace=True)

In [ ]:
#Plotting all the models
ax = train["y"].plot()
ax.plot(test["y"])
ax.plot(arima_exog_pred)
ax.plot(sarima_exog_pred)
ax.plot(fb_exog_pred)
ax.plot(ensemble_exog)
ax.set_xlim(date(2021,7,1),date(2021,8,25))
ax.set_ylim(-3,100)
ax.legend(["TRAIN", "TEST", "PRED_ARIMA_EXOG","PRED_SARIMA_EXOG", "PRED_FB_EXOG","ENSEMBLE_EXOG"]);

# 15 Looking for seasonal component in model

In [ ]:
#is there a seasonal component?
#Traing fb with exog variables
fb_model_exog_s = Prophet(daily_seasonality=True)
for _ in features:
    fb_model_exog_s.add_regressor(_)
fb_model_exog_s.fit(fb_train)

In [ ]:
#Lets make predictions for the model
fb_exog_future_s = fb_model_exog_s.make_future_dataframe(periods=test.shape[0])
for _ in features:
    fb_exog_future_s[_] = [val for val in vic_cases_wf_final[_]]
fb_exog_forecast_s = fb_model_exog.predict(fb_exog_future_s)
#forecast
fb_exog_forecast_s[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(test.shape[0])

In [ ]:
fb_model_exog_s.plot_components(fb_exog_forecast_s);

In [ ]:
fb_model_exog_s.plot(fb_exog_forecast_s[-100:]);

In [ ]:
fb_exog_forecast_s.set_index("ds", inplace=True)
RMSE = rmse(test["y"], fb_exog_forecast_s["yhat"][-test.shape[0]:])
print(f"RMSE in fb model is {RMSE}")

# 16 Lets forecast in the unknown future

In [ ]:
#Lets do a forecast finally
arima_model_exog=ARIMA(vic_cases_wf_final["y"], exog=vic_cases_wf_final[features],order=(1,0,5)).fit()
sarima_model_exog=SARIMAX(vic_cases_wf_final["y"], exog=vic_cases_wf_final[features],order=(3,0,1), seasonal_order=(2,0,0,7)).fit()
fb_full = pd.concat([fb_train, fb_test], axis=0, ignore_index=True)
fb_model_exog_s = Prophet(daily_seasonality=True)
fb_model_exog_s.add_regressor("lockdown")
fb_model_exog_s.add_regressor("delta")
fb_model_exog_s.fit(fb_full)

In [ ]:
#Lets make predictions for the model
fb_exog_future_s = fb_model_exog_s.make_future_dataframe(periods=test.shape[0])
for _ in features:
    fb_exog_future_s[_] = [val for val in vic_cases_wf_final[_]] + list(np.ones(test.shape[0]))
fb_exog_forecast_s = fb_model_exog.predict(fb_exog_future_s)
#forecast
fb_exog_forecast_s[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(test.shape[0])

In [ ]:
#Foreasting arima and sarima
arima_exog_future=fb_exog_forecast_s[["ds"]][-test.shape[0]:]

#Adding exogenous variables
for _ in features:
    arima_exog_future[_] = 1
arima_exog_future.set_index("ds", inplace=True)
start=vic_cases_wf_final.shape[0]
end=vic_cases_wf_final.shape[0] + test.shape[0] -1

#Forecasts
arima_exog_forecast = arima_model_exog.predict(start=start, end=end, exog=arima_exog_future[features])
sarima_exog_forecast = sarima_model_exog.predict(start=start, end=end, exog=arima_exog_future[features])

In [ ]:
#extracting fb porphet forecast and adding the ensemble model
fb_exog_forecast_s = fb_exog_forecast_s[["ds","yhat"]][-test.shape[0]:]
fb_exog_forecast_s.set_index("ds", inplace=True)
ensemble_exog = (fb_exog_forecast_s.yhat + sarima_exog_forecast + arima_exog_forecast)/3

# 17 Making predictions with our four models

In [ ]:
#Plotting all the models in uknown future
ax = vic_cases_wf_final["y"].plot()
ax.plot(arima_exog_forecast)
ax.plot(sarima_exog_forecast)
ax.plot(fb_exog_forecast_s)
ax.plot(ensemble_exog)
ax.set_xlim(date(2021,7,31),date(2021,8,30))
ax.set_ylim(-3,100)
ax.legend(["TRAIN", "FORECAST_ARIMA_EXOG","FORECAST_SARIMA_EXOG", "FORECAST_FB_EXOG_S", "FORECAST_ENSEMBLE_EXOG"]);

In [ ]:
print(sarima_exog_forecast)

In [ ]:
print(arima_exog_forecast)

In [ ]:
print(ensemble_exog)

In [ ]:
print(fb_exog_forecast_s)

# 18 Lets try advanced model called neural prophet that uses AR-NET or neural networks for timeseries, another tool developed by facebook and uses pytorch as the backend

In [ ]:
%%time
fb_neural_model = NeuralProphet(epochs=1000, n_forecasts=5, n_lags=3)
for _ in features:
    fb_neural_model.add_lagged_regressor(_)
fb_neural_metrics = fb_neural_model.fit(fb_full, freq='D')

In [ ]:
#Lets make predictions for the model
fb_neural_future = fb_neural_model.make_future_dataframe(df=fb_full, periods=test.shape[0])
for _ in features:
    fb_neural_future[_].iloc[-test.shape[0]:] = list(np.ones(test.shape[0]))
fb_neural_forecast = fb_neural_model.predict(fb_neural_future)
fb_neural_forecast.head()


In [ ]:
#Plotting the components
fb_neural_model.plot_components(fb_neural_forecast)

In [ ]:
#Making the forecast plot
fb_neural_model.plot(fb_neural_forecast);

In [ ]:
forecasts = []
for col in fb_neural_forecast.columns:
    if 'yhat' in col:
        forecasts.append([val for val in fb_neural_forecast[col] if val is not None][0])
forecast_df = fb_neural_forecast[["ds","y"]][-test.shape[0]:]
forecast_df.y = forecasts
forecast_df.set_index("ds", inplace=True)
forecast_df.head()

In [ ]:
ax = vic_cases_wf_final["y"].plot(figsize=(14,8))
ax.plot(forecast_df["y"])
ax.set_xlim(date(2021,7,31), date(2021,8,30))
ax.set_ylim(0,100)
ax.legend(["TRAIN","FORECAST"]);

# 19 Are these any better than moving average models, why should we use or not use them? 